# Graphengenerierung

Das ist die Implementierung des Papers "GraphRNN: Generating Realistic Graphs with Deep Auto-regressive Models"
Ziel ist es, ein neuronales Netzwerk zu implementieren, dass vorher mit Graphen gefüttert wird und dann neue erzeugt.
Eine Beispielanwendung wäre, organische Moleküle als Graphen aufzufassen und dann neue potenzielle Verbindungen
zu erstellen (You et al. 2018) .


Ein ungerichteter Graph $G = (V, E)$ ist definiert durch seine Knotenmenge
$V = \{v_1 , \dots , v_n \} $ und die Kantenmenge $E \subset \{(v_i , v_j ) \mid v_i , v_j \in V \} $.
Weiterhin wollen wir eine Knotenordnung $  \pi \colon V \to \mathbb{N} $ definieren, die jedem Knoten eine natürliche Zahl zuordnet und eine injektive Funktion darstellt.
Mithilfe der Knotenordnung können wir jedem Graphen $G$ durch eine Adjazenzmatrix $ A^\pi \in \mathbb{R}^{n \times n} $ darstellen, wobei der $ (k, l) $ - Eintrag gleich $ 1 $ ist, falls wenn $ \pi(v_i) = k $ und  $ \pi(v_j) = l $, dann $ (v_i, v_j) \in E $ gilt, und sonst ist der Eintrag $ 0 $. Sei $ \Pi $ die Menge aller Knotenordnungen des Graphen $ G $.

Das Ziel des Lernens generativer Modelle von Graphen ist das Lernen
eine Verteilung $p_{\text{model}} (\tilde{G}) $ über Graphen, basierend auf einer Menge von
beobachteten Graphen $\tilde{G} = \{G_1 , \dots, G_s \} $, die aus der umbekannten Verteilung $p$ kommen, wobei jeder Graph $ G_i $ eine unterschiedliche, endliche
Anzahl von Knoten und Kanten hat.



In [8]:
import networkx as nx
import numpy as np
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
from queue import Queue
from random import choice, randint

from torch import nn
import torch.optim as optim
import torch
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter
import pdb
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from copy import copy
#from data import *
import random
import matplotlib.pyplot as plt


In [2]:
## Erstellt die Daten, die wir zum Trainieren, Validieren und Testen benutzen
## Quelle: https://github.com/snap-stanford/GraphRNN/blob/master/create_graphs.py
def create(args):
### load datasets
    graphs=[]
    # synthetic graphs
    if args=='ladder':
        graphs = []
        for i in range(100, 201):
            graphs.append(nx.ladder_graph(i))
        max_prev_node = 10
    elif args=='ladder_small':
        graphs = []
        for i in range(2, 11):
            graphs.append(nx.ladder_graph(i))
        max_prev_node = 10
    elif args=='tree':
        graphs = []
        for i in range(2,5):
            for j in range(3,5):
                graphs.append(nx.balanced_tree(i,j))
        max_prev_node = 256
    elif args=='caveman':
        # graphs = []
        # for i in range(5,10):
        #     for j in range(5,25):
        #         for k in range(5):
        #             graphs.append(nx.relaxed_caveman_graph(i, j, p=0.1))
        graphs = []
        for i in range(2, 3):
            for j in range(30, 81):
                for k in range(10):
                    graphs.append(caveman_special(i,j, p_edge=0.3))
        max_prev_node = 100
    elif args=='caveman_small':
        # graphs = []
        # for i in range(2,5):
        #     for j in range(2,6):
        #         for k in range(10):
        #             graphs.append(nx.relaxed_caveman_graph(i, j, p=0.1))
        graphs = []
        for i in range(2, 3):
            for j in range(6, 11):
                for k in range(20):
                    graphs.append(caveman_special(i, j, p_edge=0.8)) # default 0.8
        max_prev_node = 20
    elif args=='caveman_small_single':
        # graphs = []
        # for i in range(2,5):
        #     for j in range(2,6):
        #         for k in range(10):
        #             graphs.append(nx.relaxed_caveman_graph(i, j, p=0.1))
        graphs = []
        for i in range(2, 3):
            for j in range(8, 9):
                for k in range(100):
                    graphs.append(caveman_special(i, j, p_edge=0.5))
        max_prev_node = 20
    elif args.startswith('community'):
        num_communities = int(args[-1])
        print('Creating dataset with ', num_communities, ' communities')
        c_sizes = np.random.choice([12, 13, 14, 15, 16, 17], num_communities)
        #c_sizes = [15] * num_communities
        for k in range(3000):
            graphs.append(n_community(c_sizes, p_inter=0.01))
        max_prev_node = 80
    elif args=='grid':
        graphs = []
        for i in range(10,20):
            for j in range(10,20):
                graphs.append(nx.grid_2d_graph(i,j))
        max_prev_node = 40
    elif args=='grid_small':
        graphs = []
        for i in range(2,5):
            for j in range(2,6):
                graphs.append(nx.grid_2d_graph(i,j))
        max_prev_node = 15
    elif args=='barabasi':
        graphs = []
        for i in range(100,200):
             for j in range(4,5):
                 for k in range(5):
                    graphs.append(nx.barabasi_albert_graph(i,j))
        max_prev_node = 130
    elif args=='barabasi_small':
        graphs = []
        for i in range(4,21):
             for j in range(3,4):
                 for k in range(10):
                    graphs.append(nx.barabasi_albert_graph(i,j))
        max_prev_node = 20
    elif args=='grid_big':
        graphs = []
        for i in range(36, 46):
            for j in range(36, 46):
                graphs.append(nx.grid_2d_graph(i, j))
        max_prev_node = 90

    elif 'barabasi_noise' in args:
        graphs = []
        for i in range(100,101):
            for j in range(4,5):
                for k in range(500):
                    graphs.append(nx.barabasi_albert_graph(i,j))
        graphs = perturb_new(graphs,p=args.noise/10.0)
        max_prev_node = 99

    # real graphs
    elif args == 'enzymes': ##TODO: Hier sind die Datensete
        graphs= Graph_load_batch(min_num_nodes=10, name='ENZYMES')
        max_prev_node = 25
    elif args == 'enzymes_small':
        graphs_raw = Graph_load_batch(min_num_nodes=10, name='ENZYMES')
        graphs = []
        for G in graphs_raw:
            if G.number_of_nodes()<=20:
                graphs.append(G)
        max_prev_node = 15
    elif args == 'protein':
        graphs = Graph_load_batch(min_num_nodes=20, name='PROTEINS_full')
        max_prev_no.de = 80
    elif args == 'DD':
        graphs = Graph_load_batch(min_num_nodes=100, max_num_nodes=500, name='DD',node_attributes=False,graph_labels=True)
        max_prev_node = 230
    elif args == 'citeseer':
        _, _, G = Graph_load(dataset='citeseer')
        G = max(nx.connected_component_subgraphs(G), key=len)
        G = nx.convert_node_labels_to_integers(G)
        graphs = []
        for i in range(G.number_of_nodes()):
            G_ego = nx.ego_graph(G, i, radius=3)
            if G_ego.number_of_nodes() >= 50 and (G_ego.number_of_nodes() <= 400):
                graphs.append(G_ego)
        max_prev_node = 250
    elif args == 'citeseer_small':
        _, _, G = Graph_load(dataset='citeseer')
        G = max(nx.connected_component_subgraphs(G), key=len)
        G = nx.convert_node_labels_to_integers(G)
        graphs = []
        for i in range(G.number_of_nodes()):
            G_ego = nx.ego_graph(G, i, radius=1)
            if (G_ego.number_of_nodes() >= 4) and (G_ego.number_of_nodes() <= 20):
                graphs.append(G_ego)
        shuffle(graphs)
        graphs = graphs[0:200]
        max_prev_node = 15

    return graphs




In [61]:
graphs = create("ladder")

Wir haben zurzeit keine Ordnung $ \pi $, noch ist der Graph in Vektorform vorhanden. Über eine Breitesuche der Nachbarn bekommen eine
nicht eindeutige Nummierung der Knoten. 
Die genaue Herangehensweiße ist:
1. Wir wählen einen zufälligen Knoten des Graphen, dem die Knotenordnung die Zahl $1$ zuordnet.
2. Wir wählen die Nachbarknoten dieses Knoten und $\pi $ ordnet ihnen jeweils eine unterschiedliche Zahl zu.
3. Wie in einer Breitensuche nehmen die Nachbarn dieser Knoten und ordnen diesen eine Zahl zu.

Wir hören auf, wenn der Graph durchlaufen ist. Damit haben wir eine Knotenordnung $ \pi $ gegeben.



Wir wollen die Kanten in eine spezielle Vektormenge überführen. Sei $ \pi $ gegeben. Wir definieren $ S_i  \in \{ 0, 1 \}^{i - 1} $, indem die j-te Eintrag $ 1 $ ist, falls $ \pi(v_i) $ und $ \pi(v_j) $ eine Kante besitzen, sonst $ 0 $. 

In [4]:
## Wandelt NX in SI - Arrays um
def get_si(graph):
    bfs_list = list()
    q = Queue(maxsize = 0)
    graph_list = list(graph.nodes())
    start_element = random.choice(graph_list)
    q.put(start_element)

    while(q.qsize() > 0):
        n = q.get()
        if not n in bfs_list:
            bfs_list.append(n)
            neighs = list(nx.all_neighbors(graph, n))
            for entry in neighs:
                    q.put(entry)

    result_si = list()
    for i_list, i_node in enumerate(bfs_list[1:]):
        neighs = list(nx.all_neighbors(graph, i_node))
        si = list()

        for i_vector_list, i_vector_node in enumerate(bfs_list[:i_list + 1]):
            if i_vector_node in neighs:
                si.append(1)
            else:
                si.append(0)
        si_np = np.asarray(si)
        result_si.append(si_np)
    return result_si

![title](arch_graphrnn.png)

Das neuronale Netzwerk besteht eigentlich aus den zwei Netzwerken $ f_{\text{trans}} $ und $ f_{\text{out}} $. $ f_{\text{trans}} $ ist ein Gated Recurrend Unit, kurz GRU, welches als Input $ S_{i - 1} $ und den Hidden Vector $ h_{i - 1} $ hat und einen neuen Vektor $ h_i $ ausgibt, also $ h_i = f_{\text{trans}} (S_{i - 1} , h_{i - 1}) $.

$ f_{\text{out}} $ nimmt dann den Hidden Vector $h_i $ und erstellt ein $ S_i $.
$ f_{\text{trans}} $ wird dabei als Kantennetzwerk betrachtet und $ f_{\text{out}} $ als Knotennetzwerk.

In (You et al. 2018) werden zwei Varianten vorgestellt: Einmal kann $ f_{\text{out}} $ ein einfaches Multi-Layer Perceptron (wie hier implementiert) sein. Diese Variante wird in (You et al. 2018) GraphRNN - S genannt. Eine bessere Variante ist ein GRU für $ f_{\text{out}} $ zu nehmen. Diese Variante wird dann GraphRNN in (You et al. 2018)  genannt. RNN meint hier ein Recurrend Neural Network.

In dem unteren Codeblock erstellen wir das eigentliche Modell. Für das Knotenerstellung benutzen wir ein GRU, während die Kantenerstellung durch MLP entsteht. Der Hidden Vector hat die 
Größe 128 und max_nodes gibt die maximale Größe aller $ S_i $ an.

SOS (kurz für start of sequence) und EOS (kurz für end of sequence) sind vordefinierte Vektoren.

SOS zeigt den Netzwerk an, dass ein neuer Graph generiert werden soll und EOS zeigt an, wenn der Graph fertig generiert ist. Wir benutzen später das EOS, um anzuzeigen, dass der Graph fertig ist.

Der Ablauf im **Trainingsprozess** besteht wie folgt. Wir haben eine Liste an Graphen, wobei die Graphen 
wieder aus geordneten Listen von $ S_i $ - Vektoren bestehen. Wir nehmen einen Graphen und iterieren über ein $i$-tes $ S_i $, 
wobei wir mit $ S_0 $ anfangen.
Am Anfang eines jeden Graphen geben wir SOS als Input in  $ f_{\text{trans}}$. Den entstandenen Hidden Vector nehmen wir als Input
für $ f_{\text{out}} $ und erhalten einen Vektor. Die Verlustfunktion ist binary_cross_entropy und nimmt als Argument diesen Vektor und $ S_0 $.
Danach wenden wir den Gradient Descent an.
Im nächsten Schritt nehmen wir als Eingabe $ S_0 $ und den vorherigen Hidden Vector für $ f_{\text{trans}}$.
Die Ausgabe nehmen wir für $ f_{\text{out}} $ und berechnen wieder den Loss mit $ S_1 $.

Solange wir nicht den EOS erreichen, nehmen wir  $ S_i $ und den vorherigen Hidden Vector und berechnen den Loss mit der Ausgabe und  $ S_{i + 1} $.
Falls der $ S_{i + 1} $ der EOS ist, beenden wir die Schleife und starten mit einem neuen Graphen von vorne.

Der Ablauf im **Generierungsprozess** besteht wie folgt.
Wir geben SOS als Input in  $ f_{\text{trans}}$. Den entstandenen Hidden Vector nehmen wir als Input
für $ f_{\text{out}} $ und erhalten einen Vektor. Diesen Vektor nehmen wir als Parameter für eine binomiale Zufallsfunktion und erhalten einen Vektor, der $ S_0 $ sein soll.
Im nächsten Schritt nehmen wir als Eingabe $ S_0 $ und den vorherigen Hidden Vector für $ f_{\text{trans}}$.
Die Ausgabe nehmen wir für $ f_{\text{out}} $ und erhalten wieder ein Vektor. Mit diesen Vektor wenden wir eine binomiale Zufallsfunktion an
und erhalten $ S_1 $.

Solange wir nicht den EOS zurück bekommen, nehmen wir  $ S_i $ und den vorherigen Hidden Vector und berechnen  $ S_{i + 1} $.
Falls der $ S_{i + 1} $ der EOS ist, beenden wir die Schleife und haben einen fertigen Graphen.

In [5]:
## Modell
class graphrnn_simple(nn.Module):
    def __init__(self, max_nodes):
        super(graphrnn_simple, self).__init__()
        self.node_network = nn.GRUCell(128 + max_nodes, 128)
        
        
        self.edge_mlp = nn.Linear(128, 500)
        self.intern_act = nn.ReLU()
        self.edge_mlp2 = nn.Linear(500, max_nodes)
        self.act = nn.Sigmoid() 
        torch.nn.init.xavier_uniform_(self.node_network.weight_ih)
        torch.nn.init.xavier_uniform_(self.node_network.weight_hh)
        torch.nn.init.xavier_uniform_(self.edge_mlp.weight)
        torch.nn.init.xavier_uniform_(self.edge_mlp2.weight)
        
    
    def forward(self, input):
        x = self.node_network(input)
        self.hidden_vec = x.clone().detach()
        x = self.edge_mlp(x)
        x = self.intern_act(x)
        x = self.edge_mlp2(x)
        
        x = self.act(x)
        return x

In [6]:
## SOS und EOS
def get_startvector(len_end): #hidden vec doch nur Nullen
    start = torch.cat(
        (torch.rand(128), torch.ones(len_end - 128))).view(1,len_end)
    return start


def get_endvector(len_end):
    start = torch.zeros(len_end).view(1,len_end)
    return start.numpy()

In [7]:
## Iterator. Y ist X der nächsten Iteration
def get_input_output_set(graph_set):
    for j in range(1, len(graph_set)):
        yield graph_set[j - 1], graph_set[j]

In [8]:
# Wandelt numpy Si in Tensor um
def si_to_tensor(si, length=124):
    si_len = si.shape[0] #funtioniert so?
    if si_len == length:
        return torch.from_numpy(si).view(1, length).float()
    si_torch = torch.from_numpy(si)
    si_torch = torch.cat(
        (si_torch, torch.zeros(length - si_len))
    ).view(1, length)
    return si_torch

In [9]:
def get_set (graphs, n_clones):
    return [get_si(random.choice(graphs)) for _ in range(n_clones)]

In [11]:
## fügt EOS und SOS hinzu
def prepare_set(graph_set, limit): 
    return [[np.ones(limit)] + g + [np.zeros(limit)]
                for g in graph_set if len(g) <= limit]

In [78]:

## generiert Graphen, basierend auf dem Model
def create_graphs(model, n_graphs, length=100):
    created_graphs = list()
    no_eos_graphs = list()
    for k in range(100):
        start = get_startvector(length + 128)
        distribution = model(start)
        
        inputV = torch.cat(
            (torch.FloatTensor(model.hidden_vec).view(1, 128), distribution.detach()), dim=1
        )
        i = 0
        hs = list()
        dss = list()
        while(True):
            hs.append(inputV)
            distribution = model(inputV)
            dss.append(torch.bernoulli(distribution))
            inputV = torch.cat(
                (torch.FloatTensor(model.hidden_vec).view(1, 128), distribution.detach()), dim=1
            )
            end = torch.from_numpy(get_endvector(distribution.shape[1]))
            i += 1

            ## Vermeidung von Endlosgenerierung
            if i > n_graphs:
                no_eos_graphs.append(dss)
                break
            if ( torch.round(distribution) == end).all():
                hs.append(inputV)
                if i > 1:
                    created_graphs.append(dss)
                    break
                    
    return created_graphs, no_eos_graphs


In [79]:
## wandelt BFS in NX Graphen um
def si_to_nx(graph):
    g = nx.Graph()
    for i, si in enumerate(graph):
        g.add_node(i)
        if i == 0:
            continue
        si_local = si[0, :i].detach().numpy()
        
        ind = np.where(si_local == 1.)[0].tolist()
        
        for connec_ind in ind:
            g.add_edge(connec_ind, i)

    return g

In [80]:
def si_to_nx_1d(graph):
    g = nx.Graph()
    for i, si in enumerate(graph):
        g.add_node(i)
        if i == 0:
            continue
 
        si_local = si[:i]
        
        ind = np.where(si_local == 1.)[0].tolist()
        
        for connec_ind in ind:
            g.add_edge(connec_ind, i)

    return g

In [81]:
## Konstanten
training_split = int(len(graphs) * 0.7)
val_split = int(len(graphs) * 0.85)

N_BFS_CLONES = 1000
N_VAL_CLONES = 100

QUANTILE_LIMIT = int((N_BFS_CLONES + 2 * N_VAL_CLONES) * 0.95)

In [82]:
training_set = get_set(graphs[:training_split], N_BFS_CLONES)
validation_set = get_set(graphs[training_split:val_split], N_VAL_CLONES)
test_set = get_set(graphs[val_split:], N_VAL_CLONES)



In [83]:
## erstellt quantile, um zu große Vektoren auszufiltern
len_set = [len(g) for g in training_set + validation_set + test_set]
# Das MLP braucht eine feste größe Eingabegröße. Wir nehmen den 95% - Quantil und verwerfen den Rest der Eingaben
quantile = sorted(len_set)[QUANTILE_LIMIT]
training_set = prepare_set(training_set, quantile)
validation_set = prepare_set(validation_set, quantile)
test_set = prepare_set(test_set, quantile)

Wir übernehmen die Validierungsmetriken von You et. al, 2018 und verweisen für weitere Details dorthin. Die Implementierung der Metriken stammt von https://github.com/snap-stanford/GraphRNN/blob/master/eval/stats.py

In [84]:
%run eval/stats.py

In [85]:
model = graphrnn_simple(quantile)

criterion = F.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.01)

Der Trainingsprozess läuft so ab, dass wir $ S_{ i - 1} $ und den Hiddenvector nehmen und als Ausgabe $ S_{ i} $ erwarten.
Am Anfang nehmen wir SOS und einen zufälligen Vektor statt  $ S_{ i - 1} $ und den Hiddenvector, und am Ende erwarten wir ein EOS.


In [ ]:
writer = SummaryWriter()
step = 0
last_validation_step = 0
VALIDATE_AFTER = 2000
val_dg = 0
val_cs = 0
interesting_models = list()
cut_training_set = 100
for epoch in range(2):
    min_loss = 2

    for i, g in enumerate(training_set[:cut_training_set]):

    
        running_loss = 0

        for i_one_graph, (si_input, si_output) in enumerate(get_input_output_set(g)):
            tensor_input = si_to_tensor(si_input, length=quantile)
            if i_one_graph == 0:
                inputV = torch.cat(
                (torch.rand(1, 128), tensor_input.detach()), dim=1
                ) 
            else:
                inputV = torch.cat(
                (torch.FloatTensor(model.hidden_vec).view(1, 128), tensor_input.detach()), dim=1
                ) 

            distribution = model(inputV)
            
            tensor_output = si_to_tensor(si_output, length=quantile)
            loss = criterion(distribution, tensor_output)

            loss.backward()
            inputV = torch.cat(
                (torch.FloatTensor(model.hidden_vec).view(1, 128), tensor_output.detach()), dim=1
            ) 

            writer.add_scalar("LOSS", loss.item(), step )
            step += 1
            print(f"graph {i} \t EPOCH: {epoch} \t loss  {loss.item():.5f} \t step: {step}", end="\r")

            if loss.item() < min_loss:
                min_loss = loss.item()
                best_model = copy(model)
        distribution = model(inputV)
        optimizer.step()
        
        model.zero_grad()
        optimizer.zero_grad()
        
        ## validation erstellt Graphen, um sie dann zu validieren
        if step - last_validation_step >= VALIDATE_AFTER:
            last_validation_step = step
            syn_graph, _ = create_graphs(model, 10, length=quantile)
            if len(syn_graph) == 0 or step == 0:
                print("\n synthectic 0 \n")
                writer.add_scalar("Degree Staats", 2, step )
                writer.add_scalar("Clustering Stats", 0, step )
                continue
            nx_generated_graphs = [si_to_nx(g) for g in syn_graph]

            val_dg = degree_stats(nx_generated_graphs, graphs[training_split:val_split])
            val_cs = clustering_stats(nx_generated_graphs, graphs[training_split:val_split])
            
            if val_cs > 0 or val_dg < 2 or len(syn_graph) > 0:
                interesting_models.append(copy(model))
            
            writer.add_scalar("Degree Staats", val_dg, step )
            writer.add_scalar("Clustering Stats", val_cs, step )
            

    
writer.close()

In [87]:
test_graphs, _ = create_graphs(model, 300, length=quantile)
nx_generated_graphs = [si_to_nx(g) for g in test_graphs]

In [ ]:

test_set = graphs[val_split:]
val_dg = degree_stats(nx_generated_graphs, test_set)
val_cs = clustering_stats(nx_generated_graphs, test_set)


In [107]:
torch.save(model, "bestmodels/best_model_tree.pt")

In [34]:
model = torch.load("bestmodels/best_model_tree.pt")

## Visualisierung

In [40]:
## Quelle: https://github.com/snap-stanford/GraphRNN/blob/master/utils.py
def draw_graph_list(G_list, row, col, fname = 'figures/test', layout='spring', is_single=False,k=1,node_size=55,alpha=1,width=1.3):
    # # draw graph view
    # from pylab import rcParams
    # rcParams['figure.figsize'] = 12,3
    plt.switch_backend('agg')
    for i,G in enumerate(G_list[:row * col]):
        plt.subplot(row,col,i+1)
        plt.subplots_adjust(left=0, bottom=0, right=1, top=1,
                        wspace=0, hspace=0)

        plt.axis("off")
        if layout=='spring':
            pos = nx.spring_layout(G,k=k/np.sqrt(G.number_of_nodes()),iterations=100)
            # pos = nx.spring_layout(G)

        elif layout=='spectral':
            pos = nx.spectral_layout(G)
        # # nx.draw_networkx(G, with_labels=True, node_size=2, width=0.15, font_size = 1.5, node_color=colors,pos=pos)
        # nx.draw_networkx(G, with_labels=False, node_size=1.5, width=0.2, font_size = 1.5, linewidths=0.2, node_color = 'k',pos=pos,alpha=0.2)

        if is_single:
            # node_size default 60, edge_width default 1.5
            nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='#336699', alpha=1, linewidths=0, font_size=0)
            nx.draw_networkx_edges(G, pos, alpha=alpha, width=width)
        else:
            nx.draw_networkx_nodes(G, pos, node_size=1.5, node_color='#336699',alpha=1, linewidths=0.2)
            nx.draw_networkx_edges(G, pos, alpha=0.3,width=0.2)

        # plt.axis('off')
        # plt.title('Complete Graph of Odd-degree Nodes')
        # plt.show()
    plt.tight_layout()
    plt.show()
    plt.savefig(fname+'.png', dpi=600)
    plt.close()

In [53]:
draw_graph_list(graphs[val_split:], 1, 1, fname="figures/test")

<ipython-input-40-5af41a2067aa>:34: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Ergebnisse


Hier werde ich die Ergebnisse der Netzwerke vorstellen. Erstmal werden die Testdaten visualisiert. Dann folgen Diagramme,
die den Verlauf der Loss - Kurve und der Validierungsmetriken Degree Stat und Cluster Stat beschreiben.
Zuletzt visualisieren wir die generierten Graphen.
"ladder" und "tree" sind Argumente der Funktion create. Auf diesen Graphen stützen sich die Untersuchungen


### Ladder

Wir haben hier 100 Graphen pro Epoche genommen.

#### Visualisierung der Testdaten


<img src="figures/ladder_test.png" width="1000">

#### Kurve Loss

<img src="diagramme/ladder_loss0720.png" width="1000">

#### Kurve Log Loss

<img src="diagramme/ladder_loss_log.png" width="1000">

#### Kurve Cluster Stat

<img src="diagramme/ladder_clustering_stats0720.png" width="1000">

#### Kurve Degree Stat

<img src="diagramme/ladder_degree_stats.png" width="1000">

#### Visualisierung der generierten Daten

<img src="figures/ladder_synth.png" width="1000">

### Tree

#### Visualisierung der Testdaten


<img src="figures/tree_test.png" width="1000">

#### Kurve Loss

<img src="diagramme/loss_tree.png" width="1000">

#### Kurve Log Loss

<img src="diagramme/loss_log_tree.png" width="1000">

#### Kurve Cluster Stat

<img src="diagramme/cs_tree.png" width="1000">

#### Kurve Degree Stat

<img src="diagramme/ds_tree.png" width="1000">

#### Visualisierung der generierten Daten

<img src="figures/tree_synth.png" width="1000">

## Ergebnisse des Papers

Im Paper wurden andere Graphen generiert als in diesem Notebook. Deswegen vergleichen wir nicht mit den Paper.

## Diskussion

Die Generierung ist definitiv ausbaufähig, wie anhand der Visualisierung feststellen können. Bei den Baumgraphen werden unzureichende Graphen generiert,während für
Leitergraphen wir schon gute Ergebnisse erzielen.
Eine Möglichkeit wäre es, statt GraphRNN simple die komplexere Variante zu nehmen. Weiterhin wäre es nützlich, länger mit besserer Hardware zu trainieren. Die Trainingshardware war ein Carbon X1 aus dem Jahr 2018 mit Intel® Core™ i7-7500U CPU @ 2.70GHz. Wir haben GraphRNN - simple, da alleine die Implementierung schon sehr zeitaufwendig war und einfacheres Modell mit schwächerer Hardware trainierbarer ist.
für
Weiterhin hatte ich mit einer Reihe weiterer Probleme zu kämpfen. Ich konnte nicht mit beliebig vielen Daten trainieren, denn wenn ein Modell zu overfittet war, neigte es
dazu, kein EOS mehr zu senden. Ausserdem empfand ich es als Widerspruch, dass im Paper nicht zwischen Generierung und Training unterschieden worden ist.
Im Paper wurde das Modell mit einem Zufallsprozess erläutert, aber verschwiegen, dass für den Trainingsprozess dieser wegfällt. Erst ein Blick in die konkrete Implementierung verstand ich diesen Unterschied.

### Quellen

- J. You et al, 2018. GraphRNN: Generating Realistic Graphs with Deep Auto-regressive Models In: ICML 2018
